<a href="https://colab.research.google.com/github/DachelMarzo/Dachel-CD/blob/main/Dachel_Hernandez_M5_U3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objetivo del Trabajo

### El objetivo Principal de este notebook es poder realizar distintos tipos de muestreos estadísticos que nos permitan sacar conclusiones de los resultados que nos arrojen y ver si tienen una coherencia con lo que buscamos

### Para ello vamos a realizar 3 tipos de muestreos


*   1- Muestreo <Aleatorio Simple>
*   2- Muestreo <Estratificado>
*   3- Muestreo <Sistematico>

### Vamos a trabajar con dos archivos .CSV de vinos White(Blancos) y Red(Tintos)

### Instalo la librería de Pyspark

Este código !pip install pyspark se utiliza para instalar la librería PySpark. pip es un gestor de paquetes de Python, y el símbolo ! al inicio de la línea en un entorno como Colab o Jupyter Notebook permite ejecutar comandos de shell directamente. PySpark es la interfaz Python para Apache Spark, un motor unificado de análisis para procesamiento de datos a gran escala.

In [7]:
!pip install pyspark

### Creo la Seccion de Spark

In [8]:
from pyspark.sql import SparkSession

### Monto el drive

In [9]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Nombro la seccion de la park Seccion y cargo los archivos csv

In [10]:
spark=SparkSession.builder.appName('Ejercicio_practico_M5T3').getOrCreate()

In [11]:
df_winequality_red=spark.read.csv('/content/drive/MyDrive/master/M5/winequality-red.csv',header=True,inferSchema=True, sep=';')

In [12]:
df_winequality_white=spark.read.csv('/content/drive/MyDrive/master/M5/winequality-white.csv',header=True,inferSchema=True, sep=';')

In [13]:
from pyspark.sql.functions import lit

df_blanco = df_winequality_white.withColumn("tipo_vino", lit("blanco"))
df_tinto  = df_winequality_red.withColumn("tipo_vino", lit("tinto"))


In [14]:
df_total = df_blanco.unionByName(df_tinto)


In [15]:
df_total.printSchema()

root
 |-- color: string (nullable = true)
 |-- fixed_acidity: double (nullable = true)
 |-- volatile_acidity: double (nullable = true)
 |-- citric_acid: double (nullable = true)
 |-- residual_sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free_sulfur_dioxide: double (nullable = true)
 |-- total_sulfur_dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)
 |-- tipo_vino: string (nullable = false)



### Muestro mis columnas

In [16]:
df_total.filter(df_total.tipo_vino == "blanco").show(5)
df_total.filter(df_total.tipo_vino == "tinto").show(5)


+-----+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+---------+
|color|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|tipo_vino|
+-----+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+---------+
|white|          5.2|            0.22|       0.46|           6.2|    0.066|               41.0|               187.0|0.99362|3.19|     0.42|    9.0|      5|   blanco|
|white|          5.7|            0.24|       0.47|           6.3|    0.069|               35.0|               182.0|0.99391|3.11|     0.46|    9.0|      5|   blanco|
|white|          6.0|            0.41|       0.23|           1.1|    0.066|               22.0|               148.0|0.99266| 3.3|     0.47|    9.0|      5|   blanco|
|whi

In [17]:
df_total.groupBy("tipo_vino").count().show()


+---------+-----+
|tipo_vino|count|
+---------+-----+
|   blanco| 4898|
|    tinto| 1599|
+---------+-----+



### Homogeinizo los datos

In [18]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

df_total=df_total.withColumn('quality',F.col('quality').cast(FloatType()))
df_total=df_total.withColumn('fixed_acidity',F.col('fixed_acidity').cast(FloatType()))
df_total=df_total.withColumn('volatile_acidity',F.col('volatile_acidity').cast(FloatType()))
df_total=df_total.withColumn('citric_acid',F.col('citric_acid').cast(FloatType()))
df_total=df_total.withColumn('residual_sugar',F.col('residual_sugar').cast(FloatType()))
df_total=df_total.withColumn('chlorides',F.col('chlorides').cast(FloatType()))
df_total=df_total.withColumn('free_sulfur_dioxide',F.col('free_sulfur_dioxide').cast(FloatType()))
df_total=df_total.withColumn('total_sulfur_dioxide',F.col('total_sulfur_dioxide').cast(FloatType()))
df_total=df_total.withColumn('density',F.col('density').cast(FloatType()))
df_total=df_total.withColumn('total_sulfur_dioxide',F.col('total_sulfur_dioxide').cast(FloatType()))
df_total=df_total.withColumn('pH',F.col('pH').cast(FloatType()))
df_total=df_total.withColumn('sulphates',F.col('sulphates').cast(FloatType()))
df_total=df_total.withColumn('alcohol',F.col('alcohol').cast(FloatType()))


df_total.printSchema()



root
 |-- color: string (nullable = true)
 |-- fixed_acidity: float (nullable = true)
 |-- volatile_acidity: float (nullable = true)
 |-- citric_acid: float (nullable = true)
 |-- residual_sugar: float (nullable = true)
 |-- chlorides: float (nullable = true)
 |-- free_sulfur_dioxide: float (nullable = true)
 |-- total_sulfur_dioxide: float (nullable = true)
 |-- density: float (nullable = true)
 |-- pH: float (nullable = true)
 |-- sulphates: float (nullable = true)
 |-- alcohol: float (nullable = true)
 |-- quality: float (nullable = true)
 |-- tipo_vino: string (nullable = false)



In [19]:
df_total.filter(df_total.tipo_vino == "blanco").show(5)
df_total.filter(df_total.tipo_vino == "tinto").show(5)

+-----+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+---------+
|color|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|tipo_vino|
+-----+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+---------+
|white|          5.2|            0.22|       0.46|           6.2|    0.066|               41.0|               187.0|0.99362|3.19|     0.42|    9.0|    5.0|   blanco|
|white|          5.7|            0.24|       0.47|           6.3|    0.069|               35.0|               182.0|0.99391|3.11|     0.46|    9.0|    5.0|   blanco|
|white|          6.0|            0.41|       0.23|           1.1|    0.066|               22.0|               148.0|0.99266| 3.3|     0.47|    9.0|    5.0|   blanco|
|whi

# Explicación de las variables 📜 📖 📚


Acidez fija: la mayoría de los ácidos presentes en el vino son fijos o no volátiles (no se evaporan fácilmente). La acidez es una característica determinada por la suma total de ácidos que contiene una muestra. Podemos cuantificarlos de forma indiferenciada (acidez total) o agrupados (acidez fija y acidez volátil). La acidez fija corresponde al conjunto de ácidos orgánicos de baja volatilidad, como los ácidos málico, láctico, tartárico o cítrico, y es inherente a las características de la muestra.

Acidez volátil: la cantidad de ácido acético en el vino, que en niveles demasiado altos puede provocar un sabor desagradable a vinagre. La acidez volátil corresponde al conjunto de ácidos orgánicos de cadena corta que se pueden extraer de la muestra mediante un proceso de destilación: ácido fórmico, ácido acético, ácido propiónico y ácido butírico.

Ácido cítrico: Presente en pequeñas cantidades, el ácido cítrico puede aportar frescura y sabor a los vinos. El ácido cítrico es un ácido orgánico débil e incoloro. Se encuentra de forma natural en los cítricos. En bioquímica, es un intermediario en el ciclo del ácido cítrico, que ocurre en el metabolismo de todos los organismos aeróbicos.

Azúcar residual: la cantidad de azúcar que queda tras la fermentación; es raro encontrar vinos con menos de 1 gramo/litro. El azúcar residual se refiere a los azúcares que quedan sin fermentar en un vino terminado. Se mide en gramos de azúcar por litro (g/l). La cantidad de azúcar residual afecta al dulzor del vino y, en la UE, el nivel de sal común (RS) está vinculado a términos específicos de etiquetado.
Vino seco: Generalmente tiene menos de 5 g/L de azúcar residual.
Vino semiseco: Usualmente se encuentra en un rango de 5 a 11 g/L.
Vino dulce: Inicia con aproximadamente 35 g/L y aumenta desde ahí.

Cloruros: la cantidad de sal presente en el vino. La mayor extracción de cloruro durante la vinificación se debe a los iones extraídos de los hollejos durante la fermentación. Por lo tanto, el jugo tinto no debe tener más de 356 mg/L de iones de cloruro para que el vino terminado no exceda el nivel legal máximo de 606 mg/L de cloruro (356 mg/L en jugo tinto x 1,7 = 606).

Dióxido de azufre libre: la forma libre de SO₂ existe en equilibrio entre el SO₂ molecular (como gas disuelto) y el ion bisulfito. ¿Qué es el dióxido de azufre libre en el vino? Los sulfitos libres son aquellos que pueden reaccionar y, por lo tanto, presentan propiedades germicidas y antioxidantes. Los sulfitos ligados son aquellos que han reaccionado (tanto reversible como irreversiblemente) con otras moléculas del medio vinícola. La suma de los sulfitos libres y ligados define la concentración total de sulfitos.

Dióxido de azufre total: cantidad de SO₂ libre y ligado; en bajas concentraciones, el SO₂ es prácticamente indetectable en el vino, pero en el caso del SO₂ libre. En pocas palabras, el dióxido de azufre total (TSO₂) es la porción de SO₂ libre en el vino más la porción ligada a otras sustancias químicas, como aldehídos, pigmentos o azúcares.

Densidad: la densidad del agua es similar a la del agua, dependiendo del porcentaje de alcohol y azúcar. ¿Cómo se mide la densidad del vino? Un hidrómetro es un instrumento que se utiliza para medir la densidad de líquidos. Consiste en un tubo de vidrio sellado con un bulbo con peso en un extremo. Los enólogos utilizan este instrumento para medir la densidad del mosto, del vino en fermentación y del vino terminado en relación con el agua pura. Esta relación se denomina gravedad específica (GE).

pH: describe la acidez o alcalinidad de un vino en una escala de 0 (muy ácido) a 14 (muy alcalino); la mayoría de los vinos se encuentran entre 3 y 4. ¿Qué es un pH alto en el vino? Los vinos con valores de pH más altos (>3,65) presentan una serie de posibles desafíos durante la vinificación y la crianza. En primer lugar, los vinos con pH alto tienen una mayor probabilidad de deterioro microbiano. Tradicionalmente, se utiliza dióxido de azufre (a menudo en forma de metabisulfito de potasio) para mantener los vinos estables durante la crianza.

Sulfatos: un aditivo del vino que puede contribuir a los niveles de gas dióxido de azufre (SO₂), que actúa como antimicrobiano. Los sulfitos del vino se encuentran de forma natural en niveles bajos en todos los vinos y son uno de los miles de subproductos químicos que se crean durante el proceso de fermentación. Sin embargo, el enólogo también añade sulfitos para preservar y proteger el vino de las invasiones de bacterias y levaduras. Para algunas personas, las alergias al azufre pueden estar asociadas con dolores de cabeza y congestión nasal después de una o dos copas de vino. Es un aditivo del vino que puede contribuir al aumento de los niveles de dióxido de azufre (SO₂), el cual actúa como antimicrobiano.

Alcohol: porcentaje de alcohol del vino.

Calidad: variable de salida (basada en datos sensoriales, puntuación entre 3 y 8).

## Muestreo Aleatorio Simple



In [20]:
total_rows = df_total.count()
sample_size = 100
sample_fraction = sample_size / total_rows

# Realizar el muestreo aleatorio simple de la variable 'alcohol'
sample_alcohol = df_total.select('alcohol').sample(False, sample_fraction, seed=42)

# Calcular la media y desviación típica de la muestra
mean_sample_alcohol = sample_alcohol.agg(F.mean('alcohol').alias('mean_sample')).collect()[0]['mean_sample']
stddev_sample_alcohol = sample_alcohol.agg(F.stddev('alcohol').alias('stddev_sample')).collect()[0]['stddev_sample']

print(f"Media del alcohol en la muestra: {mean_sample_alcohol:.2f}")
print(f"Desviación típica del alcohol en la muestra: {stddev_sample_alcohol:.2f}")

Media del alcohol en la muestra: 10.48
Desviación típica del alcohol en la muestra: 1.21


In [21]:
# Calcular la media y desviación típica de la población (dataset completo)
mean_population_alcohol = df_total.agg(F.mean('alcohol').alias('mean_population')).collect()[0]['mean_population']
stddev_population_alcohol = df_total.agg(F.stddev('alcohol').alias('stddev_population')).collect()[0]['stddev_population']

print(f"Media del alcohol en la población: {mean_population_alcohol:.2f}")
print(f"Desviación típica del alcohol en la población: {stddev_population_alcohol:.2f}")

Media del alcohol en la población: 10.49
Desviación típica del alcohol en la población: 1.19


### Comparación de la Media y Desviación Típica del Alcohol: Muestra vs. Población

**Muestra (n=100):**
*   **Media del alcohol en la muestra:** (valor calculado)
*   **Desviación típica del alcohol en la muestra:** (valor calculado)

**Población (dataset completo):**
*   **Media del alcohol en la población:** (valor calculado)
*   **Desviación típica del alcohol en la población:** (valor calculado)

**Análisis:**

Observando los resultados, podemos comparar cómo la media y la desviación típica de nuestra muestra de 100 elementos se aproximan a las estadísticas de la población completa. Un buen muestreo aleatorio simple debería producir estadísticas muestrales que estén razonablemente cerca de los parámetros poblacionales, aunque siempre habrá alguna variación debido al azar. Esta comparación nos ayuda a evaluar la representatividad de nuestra muestra y a entender la variabilidad que se puede esperar entre muestras.

Por ejemplo, si la media de la muestra está muy cerca de la media poblacional, sugiere que la muestra es un buen estimador del valor central del alcohol en todos los vinos. De manera similar, si la desviación típica de la muestra es similar a la de la población, indica que la variabilidad de la muestra es consistente con la variabilidad general de los datos.

# Explicacion del código
total_rows = df_total.count(): Esta línea calcula el número total de filas en tu DataFrame df_total, que representa toda la población de vinos.
sample_size = 100: Aquí se define el tamaño deseado de la muestra, en este caso, 100 elementos.  
sample_fraction = sample_size / total_rows: Se calcula la fracción que se debe muestrear. Por ejemplo, si tienes 1000 filas y quieres una muestra de 100, la fracción sería 0.1.  
sample_alcohol = df_total.select('alcohol').sample(False, sample_fraction, seed=42): Esta es la parte clave del muestreo. Se selecciona la columna 'alcohol' de df_total y luego se aplica el método sample(). Los parámetros significan:
False: Indica que el muestreo se realiza sin reemplazo (un elemento no se puede seleccionar más de una vez).
sample_fraction: La fracción calculada para determinar el tamaño de la muestra.
seed=42: Establece una "semilla" para el generador de números aleatorios, lo que asegura que el muestreo sea reproducible; es decir, si ejecutas el código varias veces con la misma semilla, obtendrás la misma muestra.
mean_sample_alcohol = sample_alcohol.agg(F.mean('alcohol').alias('mean_sample')).collect()[0]['mean_sample']: Calcula la media (promedio) de la columna 'alcohol' dentro de la muestra obtenida. agg() realiza la agregación, F.mean() calcula la media, alias() le da un nombre a la columna resultante, collect()[0] obtiene la primera fila del resultado (que es el valor de la media), y ['mean_sample'] extrae ese valor.
stddev_sample_alcohol = sample_alcohol.agg(F.stddev('alcohol').alias('stddev_sample')).collect()[0]['stddev_sample']: Similar al paso anterior, pero calcula la desviación típica (medida de dispersión) de la columna 'alcohol' en la muestra.
Finalmente, las dos últimas líneas print(...) muestran la media y la desviación típica calculadas para la muestra, formateadas a dos decimales para una mejor legibilidad.
En resumen, este código realiza un muestreo aleatorio simple de 100 valores de la variable 'alcohol' de tu conjunto de datos y luego calcula sus estadísticas descriptivas (media y desviación típica) para analizar la muestra.



## muestro estratificado

In [22]:
from pyspark.sql.functions import col

# Calculamos el total de filas
total_rows = df_total.count()

# Definimos el tamaño de la muestra
sample_size = 100

# Calculamos la fraccion de muestreo estratificado
overall_sampling_fraction = sample_size / total_rows

# Creamos el diccionario para cada estrato que vamos a seleccionar (color) usando para ello la fraccion calculada antes.

fractions_for_sampleBy = {
    'white': overall_sampling_fraction,
    'red': overall_sampling_fraction
}

print(f"Total de filas en el dataset: {total_rows}")
print(f"Fracción de muestreo global: {overall_sampling_fraction:.4f}")
print(f"Fracciones usadas para sampleBy por color: {fractions_for_sampleBy}")


stratified_sample = df_total.sampleBy(col('color'), fractions=fractions_for_sampleBy, seed=42)

# verificamos el conteo de la muestra estratificada
print("\nConteo del muestreo estratificado por tipo de vino:")
stratified_sample.groupBy("color").count().show()
print(f"Tamaño total de la muestra estratificada: {stratified_sample.count()}")

Total de filas en el dataset: 6497
Fracción de muestreo global: 0.0154
Fracciones usadas para sampleBy por color: {'white': 0.015391719255040789, 'red': 0.015391719255040789}

Conteo del muestreo estratificado por tipo de vino:
+-----+-----+
|color|count|
+-----+-----+
|white|   84|
|  red|   25|
+-----+-----+

Tamaño total de la muestra estratificada: 109


### Interpretación del Muestreo Estratificado

Hemos realizado un muestreo estratificado de `df_total` utilizando la columna `color` como estrato. El objetivo era obtener una muestra de tamaño total aproximadamente 100, manteniendo las proporciones originales de vinos blancos y tintos.

*   **Total de filas en el dataset:** `(valor de total_rows)`
*   **Fracción de muestreo global:** `(valor de overall_sampling_fraction)`
*   **Fracciones usadas para `sampleBy` por color:** `(valores de fractions_for_sampleBy)`

Los resultados del conteo de la muestra estratificada deberían mostrar un número de vinos blancos y tintos que, al ser divididos por el tamaño total de la muestra, se aproximen a las proporciones de la población original. Esto asegura que la muestra sea representativa en cuanto a la distribución del color del vino.

# PREGUNTAS EXTRA / OPCIONALES:


### Muestreo Sistemático

In [23]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

# Definir una columna para ordenar (por ejemplo, 'color')
# Un orden consistente es importante para el muestreo sistemático.
window_spec = Window.orderBy("color")

# Añadir un número de fila a cada registro
df_with_row_number = df_total.withColumn("row_num", row_number().over(window_spec))

# Definir el intervalo de muestreo (por ejemplo, seleccionar cada 10º registro)
sampling_interval = 10

# Realizar el muestreo sistemático inicial
# Esto selecciona aproximadamente total_rows / sampling_interval elementos
systematic_sample_filtered = df_with_row_number.filter(col("row_num") % sampling_interval == 0)

# Para asegurar exactamente 100 elementos, tomamos los primeros 100 de la muestra sistemática
systematic_sample = systematic_sample_filtered.limit(100)

# Mostrar el esquema y los primeros registros de la muestra sistemática
systematic_sample.printSchema()
systematic_sample.show()

# Contar el número de registros en la muestra
print(f"Número de registros en la muestra sistemática: {systematic_sample.count()}")

root
 |-- color: string (nullable = true)
 |-- fixed_acidity: float (nullable = true)
 |-- volatile_acidity: float (nullable = true)
 |-- citric_acid: float (nullable = true)
 |-- residual_sugar: float (nullable = true)
 |-- chlorides: float (nullable = true)
 |-- free_sulfur_dioxide: float (nullable = true)
 |-- total_sulfur_dioxide: float (nullable = true)
 |-- density: float (nullable = true)
 |-- pH: float (nullable = true)
 |-- sulphates: float (nullable = true)
 |-- alcohol: float (nullable = true)
 |-- quality: float (nullable = true)
 |-- tipo_vino: string (nullable = false)
 |-- row_num: integer (nullable = false)

+-----+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+---------+-------+
|color|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|tipo_vino|row_num|
+-----+-------------

### Interpretación del Muestreo Sistemático

El muestreo sistemático fue realizado para obtener una muestra de 100 registros del DataFrame `df_total`.

1.  **Preparación de los Datos:** Primero, se añadió una columna `row_num` a `df_total` que asigna un número secuencial a cada fila, después de ordenar los datos por la columna `color`. Esto es fundamental para el muestreo sistemático, ya que garantiza que la selección se realice sobre una secuencia ordenada.

2.  **Definición del Intervalo de Muestreo:** Se estableció un `sampling_interval` de 10. Esto significa que, idealmente, se seleccionaría cada décimo registro del conjunto de datos ordenado.

3.  **Filtrado Sistemático:** Se aplicó un filtro (`col("row_num") % sampling_interval == 0`) para seleccionar solo las filas donde el número de fila es un múltiplo del intervalo de muestreo. Esto generó un DataFrame intermedio (`systematic_sample_filtered`) que contenía todos los registros seleccionados sistemáticamente.

4.  **Asegurando el Tamaño Exacto de la Muestra:** Dado que el muestreo sistemático por sí solo puede no producir un tamaño de muestra exacto (dependiendo del tamaño total de la población y el intervalo), se utilizó `.limit(100)` sobre `systematic_sample_filtered`. Esto garantizó que la muestra final (`systematic_sample`) contuviera **exactamente 100 registros**.

**Conclusión:**

El resultado muestra un `DataFrame` con 100 filas, donde los registros provienen de la selección sistemática (cada 10mo registro en el orden definido) y luego truncados a los primeros 100 elementos. Este método es útil cuando se necesita una muestra de un tamaño fijo y se desea mantener una distribución sistemática a lo largo de los datos ordenados. La columna `row_num` en la salida confirma que los elementos se seleccionaron en múltiplos de 10, asegurando la naturaleza sistemática del muestreo.

### Calcula la media, mediana y varianza de tu muestra y la poblacional (el dataset completo


In [27]:
# Estadísticas para la muestra sistemática
sample_mean_alcohol = systematic_sample.agg(F.mean('alcohol').alias('mean')).collect()[0]['mean']
sample_median_alcohol = systematic_sample.agg(F.expr("percentile_approx(alcohol, 0.5)").alias('median')).collect()[0]['median']
sample_variance_alcohol = systematic_sample.agg(F.variance('alcohol').alias('variance')).collect()[0]['variance']

print("--- Estadísticas de la Muestra Sistemática (Alcohol) ---")
print(f"Media: {sample_mean_alcohol:.2f}")
print(f"Mediana: {sample_median_alcohol:.2f}")
print(f"Varianza: {sample_variance_alcohol:.2f}")

--- Estadísticas de la Muestra Sistemática (Alcohol) ---
Media: 10.19
Mediana: 9.90
Varianza: 0.92


In [28]:
# Estadísticas para la población (dataset completo)
population_mean_alcohol = df_total.agg(F.mean('alcohol').alias('mean')).collect()[0]['mean']
population_median_alcohol = df_total.agg(F.expr("percentile_approx(alcohol, 0.5)").alias('median')).collect()[0]['median']
population_variance_alcohol = df_total.agg(F.variance('alcohol').alias('variance')).collect()[0]['variance']

print("\n--- Estadísticas de la Población (Alcohol) ---")
print(f"Media: {population_mean_alcohol:.2f}")
print(f"Mediana: {population_median_alcohol:.2f}")
print(f"Varianza: {population_variance_alcohol:.2f}")


--- Estadísticas de la Población (Alcohol) ---
Media: 10.49
Mediana: 10.30
Varianza: 1.42


### Comparación de Estadísticas: Muestra Sistemática vs. Población (Alcohol)

**Muestra Sistemática (n=100):**
*   **Media del alcohol:** (valor calculado)
*   **Mediana del alcohol:** (valor calculado)
*   **Varianza del alcohol:** (valor calculado)

**Población (dataset completo):**
*   **Media del alcohol:** (valor calculado)
*   **Mediana del alcohol:** (valor calculado)
*   **Varianza del alcohol:** (valor calculado)

**Análisis:**

Al comparar las estadísticas de la muestra sistemática con las de la población completa para la variable 'alcohol', podemos evaluar la representatividad de nuestra muestra.

*   **Media:** Si la media de la muestra está cerca de la media poblacional, sugiere que el valor central del alcohol en la muestra es similar al de toda la población.
*   **Mediana:** La mediana nos da el valor central cuando los datos están ordenados, siendo menos sensible a valores atípicos que la media. Una mediana muestral cercana a la poblacional indica que la distribución central de la muestra es similar.
*   **Varianza:** La varianza mide la dispersión de los datos. Si la varianza de la muestra es similar a la poblacional, indica que la variabilidad de la muestra es consistente con la variabilidad general de los datos. Esto es importante para entender si la muestra captura la diversidad presente en la población.

En general, esperamos que una buena técnica de muestreo genere estadísticas muestrales que sean razonablemente aproximadas a los parámetros poblacionales. Sin embargo, siempre habrá alguna diferencia debido a la naturaleza aleatoria del muestreo.